In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from avcv.all import *
from models import get_stn
from utils.download import download_model, PRETRAINED_TEST_HYPERPARAMS
from utils.vis_tools.helpers import load_pil, save_image

model_class = 'ir_face'  # choose the class you want to use
resolution = 256  # resolution the input image will be resized to (can be any power of 2)
# image_path = 'my_image.jpeg'  # path to image you want to align



/home/anhvth8/.conda/envs/gg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_pil(path, resolution=None):
    arr = mmcv.imread(path, 0)
    arr = np.stack([arr]*3, -1)
    img = Image.fromarray(arr)
    if resolution is not None:
        img = img.resize((resolution, resolution), Image.LANCZOS)
    img = torch.tensor(np.asarray(img), device='cpu', dtype=torch.float).unsqueeze_(0).permute(0, 3, 1, 2)
    img = img.div(255.0).add(-0.5).mul(2)  # [-1, 1]
    return img  

In [4]:
class DS:
    def __init__(self, paths):
        self.paths = paths
    def __getitem__(self, idx):
        return load_pil(self.paths[idx], resolution)[0]
    def __len__(self):
        return len(self.paths)

In [5]:
# df = pd.read_csv('/data/RLDD/')

In [6]:
def unnorm(img):
    img = (img-img.min())/(img.max()-img.min())
    img = img*255
    img = img.permute([1,2,0]).cpu().numpy().astype('uint8')
    return img
    # return img.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to("cpu", torch.uint8).numpy()

In [7]:
ckpt = download_model(model_class)  # download model weights
stn = get_stn(['similarity', 'flow'], flow_size=128, supersize=resolution).to('cuda')  # instantiate STN
stn.load_state_dict(ckpt['t_ema'])  # load weights
test_kwargs = PRETRAINED_TEST_HYPERPARAMS[model_class]  # load test-time hyperparameters


In [24]:
import torch
paths = glob('/data/DMS_Drowsiness/all_video_symlink/4999d3cbd30f082fedae237fd867814b/croped_faces/*')
paths = list(sorted(paths))
ds = DS(paths[0:len(paths)//30])
dl = torch.utils.data.DataLoader(ds, 10, num_workers=10)

In [25]:
outs = []

with torch.no_grad():
    pbar = tqdm(dl, total=len(dl))
    for input_img in pbar:

        aligned_img = stn.forward(input_img.cuda(), output_resolution=resolution, **test_kwargs)  # forward pass through the STN
        # save_image(aligned_img, 'output.png', normalize=True, range=(-1, 1))  # save to disk
        aligned_img = [unnorm(_) for _ in aligned_img]
        outs.extend(aligned_img)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 301/301 [00:19<00:00, 15.16it/s]


In [26]:
images_to_video(outs, 'vis.mp4', output_size=(160,160))

2022-07-07 04:28:17.607 | INFO     | avcv.utils:images_to_video:283 - Write video, output_size: (160, 160)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 3006/3006, 845.4 task/s, elapsed: 4s, ETA:     0s

2022-07-07 04:28:21.167 | INFO     | avcv.utils:images_to_video:293 - -> /home/anhvth8/gitprojects/gangealing/vis.mp4


In [27]:
aligned_img = stn.forward(input_img.cuda(), output_resolution=resolution, **test_kwargs)  # forward pass through the STN


In [ ]:
# aligned_img.